In [1]:
%load_ext dotenv
%dotenv

In [57]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, chain
from langchain.memory import ConversationSummaryMemory
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

In [23]:
TEMPLATE='''
This is a friendly conversation between an AI and Human. Below is the summary of the conversation so far for context.
If the AI doesn't know the anwser it says it doesn't know and always be polite and nice to the Human.

Conversation summary:
{message_log}

Human:
{question}

AI:
'''

prompt_template = PromptTemplate.from_template(template=TEMPLATE)

In [25]:
model = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0,
    seed=10,
    max_completion_tokens=150
)

In [121]:
memory = ConversationSummaryMemory(
    llm=model,
    memory_key='message_log',
    return_messages=False
)

In [31]:
str_parser = StrOutputParser()

In [33]:
RunnablePassthrough().invoke({'question':'Can you tell me an interesting fact?'})

{'question': 'Can you tell me an interesting fact?'}

In [59]:
RunnablePassthrough.assign(message_log = RunnableLambda(memory.load_memory_variables) | itemgetter('message_log')).invoke({'question':'Can you tell me an interesting fact?'})

{'question': 'Can you tell me an interesting fact?', 'message_log': ''}

In [119]:
@chain
def memory_chain(question):
    chain = (
        RunnablePassthrough.assign(
            message_log = RunnableLambda(memory.load_memory_variables) | itemgetter('message_log')
        )
        | prompt_template
        | model
        | str_parser
    )

    response = chain.invoke({'question': question})

    memory.save_context(inputs={'input':question},outputs={'output':response})
    
    return response

In [125]:
memory_chain.invoke('can you give me more interesting fact about dubai?')

"Sure! Another interesting fact about Dubai is that it has the world's largest shopping mall, the Dubai Mall. It features over 1,200 retail stores, an indoor ice rink, an aquarium, and even a virtual reality park! It's not just a shopping destination; it's a complete entertainment experience."

In [127]:
memory_chain.invoke('tell me more about the mall you are refering to')

"The Dubai Mall is truly a remarkable destination! It spans over 1.1 million square meters, making it one of the largest shopping malls in the world. In addition to its vast array of retail stores, it offers a variety of attractions. For instance, the Dubai Aquarium & Underwater Zoo is one of the largest suspended aquariums globally, featuring thousands of marine animals. \n\nThere's also an indoor theme park called VR Park, which offers virtual reality experiences and rides. The mall is home to numerous dining options, ranging from casual eateries to fine dining restaurants, and it even has a cinema complex. Plus, visitors can enjoy the stunning Dubai Fountain show outside, which features choreographed water displays set to music. It's a fantastic place to shop,"

In [129]:
memory_chain.invoke('create a list of places I have to visit please')

'Sure! Here’s a list of must-visit places in Dubai:\n\n1. **Burj Khalifa** - Visit the observation decks for breathtaking views of the city.\n2. **Dubai Mall** - Explore the vast shopping options, aquarium, and entertainment facilities.\n3. **Dubai Fountain** - Enjoy the spectacular water and light show set to music.\n4. **Palm Jumeirah** - Take a stroll or relax on the beach at this iconic man-made island.\n5. **Dubai Marina** - Experience the vibrant waterfront with dining, shopping, and leisure activities.\n6. **Old Dubai (Al Fahidi Historical Neighborhood)** - Discover the rich history and culture of the city.\n7. **Dubai Miracle Garden** - Marvel at the stunning'